In [38]:
!pip install spotipy
!pip install --upgrade spotipy
!pip install spotipy pandas

In [33]:
import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from dotenv import load_dotenv


In [8]:
client_id = "d3b16b119e9547d6965785618bcf4543"  
client_secret = "30e320c3376a4383ba5a25af9bc8cb2a" 
redirect_uri = "http://localhost:8080"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

In [39]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd

# --- Configuration ---
CLIENT_ID = "d3b16b119e9547d6965785618bcf4543"
CLIENT_SECRET = "30e320c3376a4383ba5a25af9bc8cb2a"
REDIRECT_URI = "http://127.0.0.1:8080"
SCOPE = "playlist-read-private"
PLAYLIST_IDS = [
    "5hU11GCo5NWKLw8GcJrJI0",
    "your_playlist_id_2"
]

# --- Auth for playlist access (OAuth) ---
sp_oauth = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

# --- Auth for audio features (Client Credentials) ---
sp_cc = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

def chunkify(lst, size=25):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# --- Collect all track data ---
all_data = []

for playlist_id in PLAYLIST_IDS:
    playlist_info = sp_oauth.playlist(playlist_id)
    playlist_name = playlist_info["name"]
    print(f"🎧 Fetching: {playlist_name}")

    results = sp_oauth.playlist_items(playlist_id, limit=100)
    tracks = results["items"]
    track_meta = []
    track_ids = []

    for item in tracks:
        track = item["track"]
        if track and track["id"]:
            track_ids.append(track["id"])
            track_meta.append({
                "Playlist": playlist_name,
                "Title": track["name"],
                "Artist": track["artists"][0]["name"],
                "Popularity": track["popularity"],
                "ID": track["id"]
            })

    audio_features = []
    for chunk in chunkify(track_ids):
        try:
            audio_features.extend(sp_cc.audio_features(chunk))
        except Exception as e:
            print("⚠️ Error:", e)

    for meta, feat in zip(track_meta, audio_features):
        if feat:
            all_data.append({
                "Playlist": meta["Playlist"],
                "Title": meta["Title"],
                "Artist": meta["Artist"],
                "Popularity": meta["Popularity"],
                "Danceability": feat.get("danceability"),
                "Energy": feat.get("energy"),
                "Valence": feat.get("valence"),
                "Tempo": feat.get("tempo"),
                "Speechiness": feat.get("speechiness"),
                "Acousticness": feat.get("acousticness"),
                "Liveness": feat.get("liveness"),
                "Instrumentalness": feat.get("instrumentalness")
            })

# --- Save to CSV ---
df = pd.DataFrame(all_data)
df.to_csv("2021_playlist_detailed_features.csv", index=False)
print("✅ Saved: playlist_detailed_features.csv")


🎧 Fetching: 2021


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6PERP62TejQjgHu81OHxgM,37BZB0z9T8Xu7U3e65qxFy,3YJJjQPAbDT7mGpX3WtQ9A,61KpQadow081I2AsbeLcsb,7MAibcTli4IisCtbHKrGMh,748mdHapucXQri7IAO8yFK,4iJyoBOLtHqaGxP12qzhQI,7lPN2DXiMsVn7XUKtOW1CS,5HCyWlXZPP0y6Gqq8TgA20,67BtfxlNbhBmCDR2L2l8qd,6Uj1ctrBOjOas8xZXGqKk4,463CkQjx2Zk1yXoBuierM9,0gplL1WMoJ6iYaPgMCL0gX,5enxwA8aAbwZbf5qCHORXi,4RVwu0g32PAqgUiJoXsdF8,3Wrjm47oTz2sjIgck11l5e,6Im9k8u9iIzKMrmV7BWtlF,60ynsPSSKe6O3sfwRnIBRf,6PQ88X9TkUIAUIZJHW2upE,0lx2cLdOt3piJbcaXIV74f,3VqeTFIvhxu3DIe4eZVzGq,3FAJ6O0NOHQV8Mc5Ri6ENp,3UoULw70kMsiVXxW0L3A33,6tDDoYIxWvMLTdKpjFkc1B,3Ofmpyhv5UAQ70mENzB277 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=40iJIUlhi6renaREYGeIDS,4pt5fDVTg5GhEvEtlz9dKk,10hcDov7xmcRviA8jLwEaI,2Xr1dTzJee307rmrkt8c0g,5JCoSi02qi3jJeHdZXMmR8,3FeVmId7tL5YN8B7R3imoM,7hU3IHwjX150XLoTVmjD0q,5Z9KJZvQzH6PFmb8SNkxuk,0RbW8kWozrVSIGb4V13o0o,4u4NyuceXP7Uzh7XFJKCr1,4fSIb4hdOQ151TIL

⚠️ Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6PERP62TejQjgHu81OHxgM,37BZB0z9T8Xu7U3e65qxFy,3YJJjQPAbDT7mGpX3WtQ9A,61KpQadow081I2AsbeLcsb,7MAibcTli4IisCtbHKrGMh,748mdHapucXQri7IAO8yFK,4iJyoBOLtHqaGxP12qzhQI,7lPN2DXiMsVn7XUKtOW1CS,5HCyWlXZPP0y6Gqq8TgA20,67BtfxlNbhBmCDR2L2l8qd,6Uj1ctrBOjOas8xZXGqKk4,463CkQjx2Zk1yXoBuierM9,0gplL1WMoJ6iYaPgMCL0gX,5enxwA8aAbwZbf5qCHORXi,4RVwu0g32PAqgUiJoXsdF8,3Wrjm47oTz2sjIgck11l5e,6Im9k8u9iIzKMrmV7BWtlF,60ynsPSSKe6O3sfwRnIBRf,6PQ88X9TkUIAUIZJHW2upE,0lx2cLdOt3piJbcaXIV74f,3VqeTFIvhxu3DIe4eZVzGq,3FAJ6O0NOHQV8Mc5Ri6ENp,3UoULw70kMsiVXxW0L3A33,6tDDoYIxWvMLTdKpjFkc1B,3Ofmpyhv5UAQ70mENzB277:
 None, reason: None
⚠️ Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=40iJIUlhi6renaREYGeIDS,4pt5fDVTg5GhEvEtlz9dKk,10hcDov7xmcRviA8jLwEaI,2Xr1dTzJee307rmrkt8c0g,5JCoSi02qi3jJeHdZXMmR8,3FeVmId7tL5YN8B7R3imoM,7hU3IHwjX150XLoTVmjD0q,5Z9KJZvQzH6PFmb8SNkxuk,0RbW8kWozrVSIGb4V13o0o,4u4NyuceXP7Uzh7XFJKCr1,4f

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2xdwCJLfKCkat14mlgR26M,3XoflnowYPFRPOh3uFvHcm,5TnQ453LtEy08EwrIUtB1e,4saklk6nie3yiGePpBwUoc,6cx06DFPPHchuUAcTxznu9,2etHQJxIbV0soyPhelVs9Y,3aQem4jVGdhtg116TmJnHz,2pn8dNVSpYnAtlKFC8Q0DJ,6F2r4HgpJnvKDmkN6JEN6J,0LThjFY2iTtNdd4wviwVV2,0BI0hfbmqybnd3TezrDME3,1D3z6HTiQsNmZxjl7F7eoG,1QL7nSDZCwZMnbisV4KOXt,40uMIn2zJLAQhNXghRjBed,5fwSHlTEWpluwOM0Sxnh5k,0e8nrvls4Qqv5Rfa2UhqmO,6KgtcmCF9Ky68XC7ezxl3s,2sXf2JdbB2GlNju00kw9WE,6zSpb8dQRaw0M1dK8PBwQz,4qu63nuBpdn0qHUHuObEj1,2plRom0urixt6BE8t7kOhQ,0z8hI3OPS8ADPWtoCjjLl6,0eu4C55hL6x29mmeAjytzC,3nY8AqaMNNHHLYV4380ol0,0WSEq9Ko4kFPt8yo3ICd6T with Params: {} returned 403 due to None


⚠️ Error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2xdwCJLfKCkat14mlgR26M,3XoflnowYPFRPOh3uFvHcm,5TnQ453LtEy08EwrIUtB1e,4saklk6nie3yiGePpBwUoc,6cx06DFPPHchuUAcTxznu9,2etHQJxIbV0soyPhelVs9Y,3aQem4jVGdhtg116TmJnHz,2pn8dNVSpYnAtlKFC8Q0DJ,6F2r4HgpJnvKDmkN6JEN6J,0LThjFY2iTtNdd4wviwVV2,0BI0hfbmqybnd3TezrDME3,1D3z6HTiQsNmZxjl7F7eoG,1QL7nSDZCwZMnbisV4KOXt,40uMIn2zJLAQhNXghRjBed,5fwSHlTEWpluwOM0Sxnh5k,0e8nrvls4Qqv5Rfa2UhqmO,6KgtcmCF9Ky68XC7ezxl3s,2sXf2JdbB2GlNju00kw9WE,6zSpb8dQRaw0M1dK8PBwQz,4qu63nuBpdn0qHUHuObEj1,2plRom0urixt6BE8t7kOhQ,0z8hI3OPS8ADPWtoCjjLl6,0eu4C55hL6x29mmeAjytzC,3nY8AqaMNNHHLYV4380ol0,0WSEq9Ko4kFPt8yo3ICd6T:
 None, reason: None


SpotifyException: http status: 400, code: -1 - Unsupported URL / URI., reason: None

In [48]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

CLIENT_ID = "d3b16b119e9547d6965785618bcf4543"
CLIENT_SECRET = "30e320c3376a4383ba5a25af9bc8cb2a"
REDIRECT_URI = "http://127.0.0.1:8080"
SCOPE = "playlist-read-private"
PLAYLIST_IDS = ["7MyK3ZYElsc0Bkxlvbs8v0"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

all_data = []

for pid in PLAYLIST_IDS:
    playlist = sp.playlist(pid)
    playlist_name = playlist['name']
    print(f"📥 Fetching: {playlist_name}")
    
    tracks = sp.playlist_items(pid, limit=100)["items"]

    for item in tracks:
        track = item['track']
        if not track:
            continue

        song = track["name"]
        artist_name = track["artists"][0]["name"]
        artist_id = track["artists"][0]["id"]
        popularity = track["popularity"]

        # Get genres from artist
        genres = []
        if artist_id:
            try:
                artist_info = sp.artist(artist_id)
                genres = artist_info.get("genres", [])
            except Exception as e:
                print(f"⚠️ Error fetching genre for {artist_name}: {e}")
        
        all_data.append({
            "Playlist": playlist_name,
            "Song": song,
            "Artist": artist_name,
            "Popularity": popularity,
            "Genres": ", ".join(genres) if genres else "unknown"
        })

df = pd.DataFrame(all_data)
df.to_csv("2020_playlist_with_genres.csv", index=False)
print("✅ Saved to 2020_playlist_with_genres.csv")


📥 Fetching: 2020
✅ Saved to 2020_playlist_with_genres.csv


In [57]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

# --- Your Spotify Developer App Credentials ---
CLIENT_ID = "d3b16b119e9547d6965785618bcf4543"
CLIENT_SECRET = "30e320c3376a4383ba5a25af9bc8cb2a"
REDIRECT_URI = "http://127.0.0.1:8080"
SCOPE = "playlist-read-private"
PLAYLIST_IDS = ["2LEWfVqIUsbBcwLzX1GzKs"]  # Replace or add more playlist IDs

# --- Authenticate with OAuth ---
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))

all_data = []

for pid in PLAYLIST_IDS:
    playlist = sp.playlist(pid)
    playlist_name = playlist['name']
    print(f"📥 Fetching: {playlist_name}")
    
    tracks = sp.playlist_items(pid, limit=100)["items"]

    for item in tracks:
        track = item['track']
        if not track:
            continue

        # Basic track data
        song = track["name"]
        artist_name = track["artists"][0]["name"]
        artist_id = track["artists"][0]["id"]
        popularity = track["popularity"]
        explicit = track["explicit"]
        duration_ms = track["duration_ms"]
        album_name = track["album"]["name"]
        release_date = track["album"]["release_date"]

        # Fetch artist data
        genres = []
        artist_popularity = None
        artist_followers = None

        if artist_id:
            try:
                artist_info = sp.artist(artist_id)
                genres = artist_info.get("genres", [])
                artist_popularity = artist_info.get("popularity")
                artist_followers = artist_info.get("followers", {}).get("total")
            except Exception as e:
                print(f"⚠️ Error fetching artist data for {artist_name}: {e}")

        all_data.append({
            "Playlist": playlist_name,
            "Song": song,
            "Artist": artist_name,
            "Popularity": popularity,
            "Explicit": explicit,
            "Duration (ms)": duration_ms,
            "Album": album_name,
            "Release Date": release_date,
            "Artist Popularity": artist_popularity,
            "Artist Followers": artist_followers,
            "Genres": ", ".join(genres) if genres else "unknown"
        })

# --- Save to CSV ---
df = pd.DataFrame(all_data)
df.to_csv("2024_playlist_with_artist_album_info.csv", index=False)
print("✅ Saved to playlist_with_artist_album_info.csv")


📥 Fetching: 2024
✅ Saved to playlist_with_artist_album_info.csv
